In [1]:
import numpy as np
import sympy as sp
from sympy import Symbol
from sympy import solve, Poly, Eq, Function, exp
import math
import matplotlib.pyplot as plt
from numpy.lib.scimath import sqrt as csqrt
from scipy.signal import find_peaks
from scipy.constants import lambda2nu, nu2lambda, c
import os
import imp
import matplotlib.pyplot as plt
import sys
import pickle as plk
import pandas as pd
# Moises Path

#windows path
os.add_dll_directory('C:\\Program Files\\Lumerical\\v202\\api\\python\\') 
lumapi = imp.load_source('lumapi','C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')

#linux path
#sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
#import lumapi
#sys.path.append(os.path.dirname('/home/moises/Downloads/'))

In [2]:
mode = lumapi.MODE(filename = 'Edge_Coupler.lms')

In [37]:
W = 140e-9
G = 400e-9
h = 220e-9
Wr = 0.75e-6
W_MMI = 1.6e-6
G_MMI = 0.08e-6
Wl = 0.45e-6
L = 40e-6
L_MMI = 5.1e-6
L_Taper = L - L_MMI
core = 'Si (Silicon) - Palik'

In [48]:
mode.switchtolayout()
mode.select('mmi_input_up')
mode.delete()
mode.select('mmi_input_down')
mode.delete()
mode.select('mmi_output')
mode.delete()
mode.select('MMI BOX')
mode.delete()
mode.select('output')
mode.delete()

mode.addobject('linear_taper')
mode.set({
    'name':'mmi_input_up',
    'x':L_Taper + 0.75e-6,
    'y':G_MMI/2 + 0.375e-6,
    'z':0,
    'width_l':Wl,
    'width_r':Wr,
    'angle_side':90,
    'len':1.5e-6,
    'thickness':h
})

mode.addobject('linear_taper')
mode.set({
    'name':'mmi_input_down',
    'x':L_Taper + 0.75e-6,
    'y':-G_MMI/2 - 0.375e-6,
    'z':0,
    'width_l':Wl,
    'width_r':Wr,
    'angle_side':90,
    'len':1.5e-6,
    'thickness':h
})

mode.addrect()
mode.set({
    'name':'MMI BOX',
    'x':L_Taper + 1.5e-6 + 1.05e-6,
    'x span':2.1e-6,
    'y':0,
    'y span':1.6e-6,
    'z':0,
    'z span':h,
    'material':core
})

mode.addobject('linear_taper')
mode.set({
    'name':'mmi_output',
    'x':L_Taper + 3.6e-6 + 0.75e-6,
    'y':0,
    'z':0,
    'width_l':Wr,
    'width_r':Wl,
    'angle_side':90,
    'len':1.5e-6,
    'thickness':h
})

mode.addrect()
mode.set({
    'name':'output',
    'x':L + 0.5e-6,
    'x span':1e-6,
    'y':0,
    'y span':0.45e-6,
    'z':0,
    'z span':h,
    'material':core
})

